In [1]:
# Task 1:
    # Complete the following steps to establish a connection pool: 
        # To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
        # To find the information on the error, import the Error class from MySQL Connector/Python.
        # Define your database configurations as a Python dictionary object called dbconfig.
        # Establish a connection pool [pool_name = pool_b] with two connections. 
        # Implement error handling using a try-except block in case the connection fails. 

In [10]:
#Import required modules
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

#Make configuration data
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": ""
}

#Try except block 
try:
    
    #Create and get connection
    pool = MySQLConnectionPool(pool_name = "pool_b", pool_size=2, **dbconfig)
    conn = pool.get_connection()

    #Verify connection
    if conn.is_connected:
        #Success
        print("Conectado")
    else:
        #Getting wrong
        print("Fallo al establecer la conexión")

    #Closing connection
    conn.close()

#Get and manipulare exceptions
except Error as e:
    print("Error: {} -> {}".format(e.errno, e.msg))

Conectado


In [ ]:
#Task 2:
    #Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:
    #TIP: You need to add a connection to connect the third guest.

In [11]:
#Import required modules
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector

#Make configuration data
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": ""
}

#Try except block 
try:
    
    #Invitados
    guests = [
        {"table_number": 8, "first_name": "Anees", "last_name": "Java", "booking_time": "18:00", "employeeID": 6},
        {"table_number": 5, "first_name": "Bald", "last_name": "Vin", "booking_time": "19:00", "employeeID": 6},
        {"table_number": 12, "first_name": "Jay", "last_name": "Kon", "booking_time": "19:30", "employeeID": 6}, 
    ]
    #Se crea la conexion
    pool = MySQLConnectionPool(pool_name = "pool_b", pool_size=2, **dbconfig)
    
    print("Conectado, realizando peticiones")
    print("El tamaño del pool inicial es: {}".format(pool.pool_size))

    #Recorrer a los invitados
    for guest in guests:
        try:
            #Obtener conexion por empleado y mostrar datos
            guest_connected = pool.get_connection()            
            print("Table Number {}, First Name {}, Last Name {}, Booking Time {}, EmployeeID {} se conecto."
                  .format(guest["table_number"], guest["first_name"], guest["last_name"], guest["booking_time"], guest["employeeID"]))
        except:        
            #En caso de errar generar una nueva conexion
            print("Ya no hay conexiones libres, creando una.")
            
            connection = connector.connect(user=dbconfig["user"],password=dbconfig["password"])
            pool.add_connection(cnx=connection)
            user_connected = pool.get_connection()
            
            print("Extra connection: Table Number {}, First Name {}, Last Name {}, Booking Time {}, EmployeeID {} se conecto."
                  .format(guest["table_number"], guest["first_name"], guest["last_name"], guest["booking_time"], guest["employeeID"]))

    print("El tamaño del pool quedo en: {}".format(pool.pool_size))
    #Cerrar la conexion
    connection.close()

except Error as e:
    print("Error: {} -> {}".format(e.errno, e.msg))

Conectado, realizando peticiones
El tamaño del pool inicial es: 2
Table Number 8, First Name Anees, Last Name Java, Booking Time 18:00, EmployeeID 6 se conecto.
Table Number 5, First Name Bald, Last Name Vin, Booking Time 19:00, EmployeeID 6 se conecto.
Ya no hay conexiones libres, creando una.
Extra connection: Table Number 12, First Name Jay, Last Name Kon, Booking Time 19:30, EmployeeID 6 se conecto.
El tamaño del pool quedo en: 2


In [15]:
#Taks 3 and 4 in the same one

#Task 3:
    #Create a report containing the following information:
        #The name and EmployeeID of the Little Lemon manager.
        #The name and role of the employee who receives the highest salary.
        #The number of guests booked between 18:00 and 20:00.
        #The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
                
#Task 4:
    #Create a stored procedure named BasicSalesReport that returns the following statistics: 
        #Total sales
        #Average sale
        #Minimum bill paid
        #Maximum bill paid
#
#delimiter //
#CREATE PROCEDURE IF NOT EXISTS BasicSalesReport ()
#BEGIN
#	SELECT 
#		count(BillAmount) count_sales,
#		SUM(BillAmount) total_sales,
#		AVG(BillAmount) avg_sales,
#		MIN(BillAmount) min_sale,
#		Max(BillAmount) max_sale
#		FROM orders;
#END // 

In [16]:
#Import required modules
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

#Make configuration data
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": ""
}

#Function to create sql query 
def make_query(table = '', fields = ['*'], where_condition = '', order_by = '', limit = ''):
    #Make a ternary operator for query
    where_condition_c = " where {}".format(where_condition) if where_condition != '' else ''
    order_by_c = " order by {}".format(order_by) if order_by != '' else ''
    limit_c = " limit {}".format(limit) if limit != '' else ''
    fields_c = ', '.join(map(str, fields))
    #return all the query
    return "SELECT {} from {} {} {} {}".format(fields_c, table, where_condition_c, order_by_c, limit_c)
    

try:
    pool = MySQLConnectionPool(pool_name = "pool_b", pool_size=2, **dbconfig)
    print("Conectado, realizando peticiones")

    print("+-----------------------------+")

    connection = pool.get_connection()

    if connection.is_connected:
        cursor = connection.cursor()
        
        print("################")
        print("Primer sentencia: ")
        cursor.execute(
            make_query("employees", ['EmployeeID', 'Name'], 'Role = "Manager"')
        )
        data = cursor.fetchall()
        
        print("El manager {} de Little Lemon tiene el id: {}".format(data[0][1], data[0][0]))

        print("################")
        print("Segunda sentencia: ")
        cursor.execute(
            make_query('employees', ['Name', 'Role', 'Annual_Salary'], '', 'Annual_Salary desc', 1)
        )
        data = cursor.fetchall()        
        print("El {} {} es el que gana mas: {}".format(data[0][1], data[0][0], data[0][2]))

        print("################")
        print("Tercer sentencia: ")
        cursor.execute(
            make_query('bookings', ['count(*)'], ' hour(BookingSlot) between 18 and 20 ')
        )
        data = cursor.fetchall()        
        print("Hubo {} reservas entre las 18 y 20 horas hoy.".format(data[0][0]))

        print("################")
        print("Cuarta sentencia: ")
        cursor.execute(
            make_query(
                'bookings', 
                ['concat(GuestFirstName, " ", GuestLastName)', 'BookingID', 'BookingSlot'], 
                '', 'BookingSlot asc')
        )
        data = cursor.fetchall()        
        for i in range(len(data)):
            print("Cliente {} con id {} esperando desde las {}.".format(data[i][0], data[i][1], data[i][2]))

        print("################")
        print("Quinta tarea, llamar procedimiento BasicSalesReport: ")

        cursor.callproc('BasicSalesReport')
        res = next(cursor.stored_results())

        data = res.fetchall()

        print("Hubo {} ventas generando {} pesos, promedio: {}, venta minima: {}, maxima: {}"
            .format(data[0][0], data[0][1], data[0][2], data[0][3], data[0][4]))

    else:
        print("No se puede establecer conexión")
            
    connection.close()

except Error as e:
    print("Error: {} -> {}".format(e.errno, e.msg))

Conectado, realizando peticiones
+-----------------------------+
################
Primer sentencia: 
El manager Mario Gollini de Little Lemon tiene el id: 1
################
Segunda sentencia: 
El Manager Mario Gollini es el que gana mas: $70,000
################
Tercer sentencia: 
Hubo 4 reservas entre las 18 y 20 horas hoy.
################
Cuarta sentencia: 
Cliente Vanessa McCarthy con id 3 esperando desde las 15:00:00.
Cliente Marcos Romero con id 4 esperando desde las 17:30:00.
Cliente Hiroki Yamane con id 5 esperando desde las 18:30:00.
Cliente Anna Iversen con id 1 esperando desde las 19:00:00.
Cliente Joakim Iversen con id 2 esperando desde las 19:00:00.
Cliente Diana Pinto con id 6 esperando desde las 20:00:00.
################
Quinta tarea, llamar procedimiento BasicSalesReport: 
Hubo 5 ventas generando 243 pesos, promedio: 48.6000, venta minima: 37, maxima: 86


In [ ]:
#Task 5:
    #Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:
        #Get a connection from the pool.
        #Create a buffered cursor.
        #Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 
        #Returned the connection back to the pool.
        
        #The output should be as follows:
            #[BookingSlot]
            #[Guest_name]
            #[Assigned to: Employee Name [Employee Role]]

In [13]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector

dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": ""
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b", pool_size=2, **dbconfig)
    
    print("Conectado, realizando peticiones")
    try:
        conn = pool.get_connection()
        if conn.is_connected:
            cursor = conn.cursor(buffered = True)

            cursor.execute("""
                SELECT b.BookingSlot, concat(b.GuestFirstName, " ", b.GuestLastName), e.Name, e.Role from bookings b left join employees e on b.EmployeeID = e.EmployeeID limit 3
            """)

            data = cursor.fetchall()
            for i in range(len(data)):
                print("{} {} Asignado al {} {}".format(data[i][0], data[i][1], data[i][3], data[i][2]))

    except:        
        print("Ya no hay conexiones libres.")
    conn.close()
except Error as e:
    print("Error: {} -> {}".format(e.errno, e.msg))

Conectado, realizando peticiones
19:00:00 Anna Iversen Asignado al Manager Mario Gollini
19:00:00 Joakim Iversen Asignado al Manager Mario Gollini
15:00:00 Vanessa McCarthy Asignado al Head Chef Giorgos Dioudis
